# Aula 08 - Aprendendo a Ranquear - Exercícios

In [140]:
import pandas as pd
import numpy as np
import random

### Importar base de dados

In [141]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz


Saved under ml-20m-compact.tar (35).gz


x dataset/
x dataset/tags_sample.csv
x dataset/._.DS_Store
x dataset/.DS_Store
x dataset/movies_sample.csv
x dataset/._genome-tags.csv
x dataset/genome-tags.csv
x dataset/._ml-youtube.csv
x dataset/ml-youtube.csv
x dataset/._genome-scores.csv
x dataset/genome-scores.csv
x dataset/ratings_sample.csv


In [142]:
movies = pd.read_csv('./dataset/movies_sample.csv', names=['itemId', 'title', 'genre'], header=0)
ratings = pd.read_csv('./dataset/ratings_sample.csv', names=['userId', 'itemId', 'rating', 'timestamp'], header=0)
df = ratings[['userId', 'itemId', 'rating']]
df = df.merge(movies[['itemId', 'title']])
df

,userId,itemId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


### Mapeamento de ids

In [143]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.itemId.unique())}
df['userId'] = df['userId'].map(map_users)
df['itemId'] = df['itemId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.itemId] = row.title


### Divisão da base em treino e teste

In [144]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

***Exercício 01:*** Comparação entre um algoritmo pair-wise e point-wise:
- Utilizando o BPR implementado em aula, gere uma lista de 10 recomendações para um determinado usuário da base.
- Calcule a precisão média dessas recomendações usando como ground-truth o conjunto de teste. 
- Utilize o algoritmo MatrixFactorization (SVD otimizado) de predição de notas para prever as notas dos itens que aquele usuário não avaliou ainda. 
- Ordene inversamente esses itens pela nota predita, e recomende os 10 primeiros filmes.
- Calcule a precisão média dessas recomendações e compare o resultado com o resultado obtido pelo BPR. 

In [172]:
# TODO

# Get the list of all items a user has rated
def get_item_ids(df, userId):
    if userId not in df['userId'].values:
        return []
    return (df.loc[(df.userId == userId), 'itemId'].tolist())

# Dictionaries of seen and unseen movies of each user
observed = dict()
unobserved = dict()
all_users = df['userId'].unique().tolist() 
all_items = df['itemId'].unique().tolist() 
for u in all_users:
    observed[u] = get_item_ids(train, u)
    unobserved[u] = list(set(all_items) - set(observed[u]))


# Function to train a Bayesian Personalized Ranking Matrix Factorization (BPRMF) model
def draw(userId):    
    # Randomly select an observed item (i) and an unobserved item (j) for a given user
    i = random.choice(observed[userId])
    j = random.choice(unobserved[userId])
    return i, j

# Train BPRMF model
def train_bprmf(train, n_factors, lr=0.05, reg=0.02, miter=30):    
    # Get the number of unique users and items in the dataset
    n_users = df['userId'].max() + 1
    n_items = df['itemId'].max() + 1

    # Initialize item bias vector with zeros
    item_bias = np.zeros(n_items)

    # Initialize user and item latent factor matrices with small random values from a normal distribution
    p = np.random.normal(0, 0.1, (n_users, n_factors))
    q = np.random.normal(0, 0.1, (n_items, n_factors))

    # List to store the cumulative error after each iteration
    error = []

    # Loop over the number of iterations specified (miter) to update the model parameters
    for t in range(miter):
        # Variable to accumulate the ranking error
        sq_error = 0

        # Randomly select users to process in this iteration
        random_users = random.choices(train['userId'].unique(), k=len(train))

        # Loop through each randomly selected user
        for u in random_users:
            i, j = draw(u)

            # Compute the difference in predicted preferences between item i (observed) and item j (unobserved)
            x_uij = item_bias[i] - item_bias[j] + (np.dot(p[u], q[i]) - np.dot(p[u], q[j]))

            # Accumulate the ranking error
            sq_error += x_uij

            # Compute the logistic function (sigmoid) of x_uij
            eps = 1 / (1 + np.exp(x_uij))

            # Update item biases
            item_bias[i] += lr * (eps - reg * item_bias[i])  
            item_bias[j] += lr * (-eps - reg * item_bias[j]) 

            # Adjust the factors
            u_f = p[u]
            i_f = q[i]
            j_f = q[j]

            # Compute and apply factor updates
            p[u] += lr * ((i_f - j_f) * eps - reg * u_f) 
            q[i] += lr * (u_f * eps - reg * i_f)  
            q[j] += lr * (-u_f * eps - reg * j_f)  

        # Append the average error for this iteration to the error list
        error.append(sq_error / len(random_users))

    return item_bias, p, q, error

In [185]:
b, p, q, error = train_bprmf(train, 4)

In [201]:
# Function to predict top N recommended items for each user
def predict(b, p, q, N=10):
    w = b.T + np.dot(p, q.T)
    ranking = []
    
    for u, user in enumerate(all_users):
        partial_ranking = list()
        candidate_items = sorted(range(len(w[u])), key=lambda k: w[u][k], reverse=True)
        
        for i in candidate_items:
            if i not in observed[user]:
                partial_ranking.append((user, i, w[u][i]))

            if len(partial_ranking) == N:
                break

        ranking += partial_ranking
        
    return pd.DataFrame(ranking, columns=['userId', 'movieId', 'score'])

In [202]:
ranking = predict(b, p, q)

In [203]:
ranking['title'] = ranking.movieId.map(map_title)
ranking[ranking['userId'] == 0]

,userId,movieId,score,title
0,0,108,2.448924,Dogville (2003)
1,0,43,2.147719,Charlotte's Web (1973)
2,0,33,2.084482,North by Northwest (1959)
3,0,29,1.915890,"Man in the Iron Mask, The (1998)"
4,0,17,1.898968,Chasing Amy (1997)
5,0,28,1.897813,My Best Friend's Wedding (1997)
6,0,25,1.770681,Keeping the Faith (2000)
7,0,13,1.721437,Sliding Doors (1998)
8,0,5,1.720912,"I, Robot (2004)"
9,0,21,1.713398,Conspiracy Theory (1997)


In [162]:
# Calculate the average precision for a user
def average_precision(user_id, recommendations, test): 
    # Get the recommendations for the specific user
    recs_user = recommendations.loc[(recommendations.userId == user_id), 'title'].tolist()
    # Get the relevant movies for the specific user
    relevant_movies = test.loc[(test.userId == user_id), 'title'].tolist()
    
    n_relevant_movies = 0
    cumulative_precision = 0.0
    
    # Iterate over the recommendations and calculate the precision
    for i, movie in enumerate(recs_user):
        if movie in relevant_movies:
            n_relevant_movies += 1
            # Precision at index i
            precision_at_i = n_relevant_movies / (i + 1)
            cumulative_precision += precision_at_i
    
    if n_relevant_movies == 0:
        return 0.0
    
    ap = cumulative_precision / n_relevant_movies
    return ap

In [204]:
# Average precision for userId == 0 using bprmf
u = 0
recommendations = ranking[ranking['userId'] == 0]
print(f"User: {u}, AP = {average_precision(u, recommendations, test):.4f}") 

User: 0, AP = 0.1111


In [170]:
from math import sqrt

# Function to train an SVD model optimized
def train_svdopt(train, n_factors, lr=0.05, reg=0.02, miter=30):
    # Calculate the global mean rating across all users and items in the training set
    global_mean = train['rating'].mean()

    # Get the number of unique users and items from the dataset
    n_users = df['userId'].max() + 1
    n_items = df['itemId'].max() + 1  

    # Initialize user and item bias vectors with zeros
    bu = np.zeros(n_users)
    bi = np.zeros(n_items)

    # Initialize user and item latent factor matrices with small random values from a normal distribution
    p = np.random.normal(0.1, 0.1, (n_users, n_factors))
    q = np.random.normal(0.1, 0.1, (n_items, n_factors))

    # List to store the root mean squared error (RMSE) after each iteration
    error = []

    # Loop over the number of iterations specified (miter) to update the model parameters
    for t in range(miter):
        # Variable to accumulate the squared error for this iteration
        sq_error = 0

        # Loop through each user-item-rating triplet in the training data
        for index, row in train.iterrows():
            u = row['userId'] 
            i = row['itemId'] 
            r_ui = row['rating']  # Actual rating given by user u to item i

            # Compute the predicted rating based on the model
            pred = global_mean + bu[u] + bi[i] + np.dot(p[u], q[i])

            # Calculate the error between the actual and predicted rating
            e_ui = r_ui - pred

            # Accumulate the squared error for RMSE calculation
            sq_error += pow(e_ui, 2)

            # Update the user and item biases with the gradient of the error
            bu[u] += lr * e_ui 
            bi[i] += lr * e_ui

            # Update the latent factors for both the user and the item
            for f in range(n_factors):
                temp_uf = p[u][f] 
                p[u][f] += lr * (e_ui * q[i][f] - reg * p[u][f]) 
                q[i][f] += lr * (e_ui * temp_uf - reg * q[i][f]) 

        # Append the root mean squared error (RMSE) for this iteration to the error list
        error.append(sqrt(sq_error / len(train)))

    return global_mean, bu, bi, p, q, error


In [198]:
global_mean, bu, bi, p, q, error = train_svdopt(train, 4)

In [196]:
import pandas as pd

# Function that returns the rating prediction of the unseen movies of a user
def get_unseen_movies_pred_ratings(unobserved_movies, user_id, df):
    # Get the list of unseen movies for the user
    user_unobserved_movies = unobserved_movies[user_id]
    
    # Create an empty list to hold the prediction data
    predictions = []

    # Iterate over each unseen movie and calculate the predicted rating
    for movie_id in user_unobserved_movies:
        pred_rating = global_mean + bu[user_id] + bi[movie_id] + np.dot(p[user_id], q[movie_id])

        title = df[df['itemId'] == movie_id]['title'].values[0]

        # Append the userId, movieId, title, and predicted rating to the list
        predictions.append([user_id, movie_id, title, float(pred_rating)])
    
    # Convert the list into a DataFrame with columns: 'userId', 'itemId', 'pred_rating'
    pred_ratings_df = pd.DataFrame(predictions, columns=['userId', 'itemId', 'title',  'pred_rating'])
    
    return pred_ratings_df

# Get predicted ratings of unseen movies for userId == 0
pred_ratings_unseen_movies = get_unseen_movies_pred_ratings(unobserved, 0, df)

# Sort the df by the 'pred_rating' column in descending order
top_10_ratings = pred_ratings_unseen_movies.sort_values(by='pred_rating', ascending=False).head(10)

top_10_ratings

,userId,itemId,title,pred_rating
257,0,268,Napoléon (1927),6.233752
258,0,269,"Boy Named Charlie Brown, A (1969)",5.878945
180,0,191,Cockfighter (1974),5.584241
118,0,129,Interstellar (2014),5.487810
113,0,124,Into the Woods (1991),5.463041
206,0,217,Wish I Was Here (2014),5.391490
143,0,154,First on the Moon (Pervye na Lune) (2005),5.376967
299,0,310,Watch Out for the Automobile (Beregis avtomobi...,5.356144
142,0,153,Eight Deadly Shots (Kahdeksan surmanluotia) (1...,5.338202
350,0,361,Citizen Koch (2013),5.291135


In [200]:
# Average precision for userId == 0 using matrix factorization
u = 0
print(f"User: {u}, AP = {average_precision(u, top_10_ratings, test):.4f}") 

User: 0, AP = 0.0000


***Exercicio 02:*** Na aula vimos a implementação do método draw() do algoritmo BPR tradicional. Para um dado usuário, este método retorna aleatoriamente um item que ele viu (**item i**) e um outro item que ele não conhece (**item j**). O algoritmo assume que o item visto é preferível ao que ele não viu, e isso é usado para maximizar a diferença entre os scores desses itens (veja a variável **x_uij** da implementação). 

Um problema dessa abordagem é que os itens não vistos são necessariamente encarados como menos relevante pelo algoritmo, o que nem sempre acontece pois pode ser que o usuário não tenha interagido com aquele item pois não o conhece, mas que poderia gostar. Um outro problema é quando ambos os itens i e j foram vistos pelo usuário, o que numa estratégia com feedback exclusivamente implícito o BPR não consegue diferenciar qual item é preferível ao usuário. 

Você consegue pensar numa estratégia aperfeiçoada para o método draw()? Exemplos:
- Se tivermos acesso aos metadados, podemos construir um perfil para o usuário de modo que o método draw() vai retornar itens j que estão mais distantes desse perfil.
- Se dois itens i e j foram vistos, use os metadados (ou as notas, caso use feedback explícito) para decidir qual deles deve ser o i e qual deve ser o j. 
- Etc.

Implemente pelo menos uma estratégia de aperfeiçoamento do BPR, e compare com a versão original. 

In [248]:
# TODO

# Original version of draw()
def draw_original(userId):
    i = random.choice(observed[userId])
    j = random.choice(unobserved[userId])
    return i, j

# Improved version of draw() using explicit feedback
def draw_improved(userId, df):
    # Select the movie with the highest rating of all seen movies
    i = max(observed[userId], key=lambda x: df.loc[df['itemId'] == x]['rating'].values[0])

    # Select a random unseen movie
    j = random.choice(unobserved[userId])

    return i, j


In [249]:
# Train BPRMF model
def train_bprmf(train, n_factors, draw_method, lr=0.05, reg=0.02, miter=10):    
    # Get the number of unique users and items in the dataset
    n_users = df['userId'].max() + 1
    n_items = df['itemId'].max() + 1

    # Initialize item bias vector with zeros
    item_bias = np.zeros(n_items)

    # Initialize user and item latent factor matrices with small random values from a normal distribution
    p = np.random.normal(0, 0.1, (n_users, n_factors))
    q = np.random.normal(0, 0.1, (n_items, n_factors))

    # List to store the cumulative error after each iteration
    error = []

    # Loop over the number of iterations specified (miter) to update the model parameters
    for t in range(miter):
        # Variable to accumulate the ranking error
        sq_error = 0

        # Randomly select users to process in this iteration
        random_users = random.choices(train['userId'].unique(), k=len(train))

        # Loop through each randomly selected user
        for u in random_users:
            if draw_method == 'original':
                i, j = draw_original(u)
            elif draw_method == 'improved':
                i, j = draw_improved_popularity(u, df)


            # Compute the difference in predicted preferences between item i (observed) and item j (unobserved)
            x_uij = item_bias[i] - item_bias[j] + (np.dot(p[u], q[i]) - np.dot(p[u], q[j]))

            # Accumulate the ranking error
            sq_error += x_uij

            # Compute the logistic function (sigmoid) of x_uij
            eps = 1 / (1 + np.exp(x_uij))

            # Update item biases
            item_bias[i] += lr * (eps - reg * item_bias[i])  
            item_bias[j] += lr * (-eps - reg * item_bias[j]) 

            # Adjust the factors
            u_f = p[u]
            i_f = q[i]
            j_f = q[j]

            # Compute and apply factor updates
            p[u] += lr * ((i_f - j_f) * eps - reg * u_f) 
            q[i] += lr * (u_f * eps - reg * i_f)  
            q[j] += lr * (-u_f * eps - reg * j_f)  

        # Append the average error for this iteration to the error list
        error.append(sq_error / len(random_users))

    return item_bias, p, q, error


# Calculate MAP
def calculate_map(predictions, train_data):
    n_users = predictions['userId'].nunique()
    map_sum = 0

    # For each user, calculate the mean average precision
    for user_id in predictions['userId'].unique():
        relevant_items = set(train_data[train_data['userId'] == user_id]['itemId'])
        recommended_items = predictions[predictions['userId'] == user_id]
        
        n_relevant = 0
        precision_sum = 0
        
        # For each position in the ranking, check if the item is relevant
        for i, row in enumerate(recommended_items.itertuples(), 1):
            if row.movieId in relevant_items:
                n_relevant += 1
                precision_sum += n_relevant / i
        
        if len(relevant_items) > 0:
            map_sum += precision_sum / len(relevant_items)
    
    return map_sum / n_users


# Function to train and evaluate the BPR with mean average precision metric
def train_and_evaluate(train_data, draw_method, n_factors):
    # Train the BPRMF model
    item_bias, p, q, error = train_bprmf(train_data, n_factors, draw_method)

    
    # Generate rankings and predictions
    predictions = predict(item_bias, p, q)
    # Evaluate performance using MAP
    map_score = calculate_map(predictions, train_data)

    return map_score


map_original = train_and_evaluate(df, 'original', 4)
print(f"MAP - Original: {map_original:.4f}")

map_improved = train_and_evaluate(df, 'improved', 4)
print(f"MAP - Improved: {map_improved:.4f}")

MAP - Original: 0.0520
MAP - Improved: 0.0041
